# Distribució de gènere en els òrgans de govern de les AFAs de Catalunya

En aquest document s'analitza la distribució de gènere entre els càrrecs dels òrgans de govern de les
Assemblees de Famílies d'Alumnes de Catalunya.

En concret es calcula, a partir del llistat complet de càrrecs en òrgans de govern de les AFAs registrades a Catalunya amb el gènere associat al nom, el percentage de:

- Dones i homes amb càrrecs en òrgans de govern
- D'AFAs amb òrgans 100% femenins
- D'AFAs amb òrgans 100% masculins
- D'AFAs equitatives, amb una partitat entre el 40 i el 60%
- D'AFAs amb més dones que homes a l'òrgan de govern
- D'AFAs amb presidència femenina

Tots aquests valors es calculen a nivell global de tot Catalunya, per província i per comarca.

Dades originals obtingudes al portals de [Transparència](https://www.transparenciacatalunya.cat/) i
[Dades Obertes](https://analisi.transparenciacatalunya.cat/) de la Generalitat de Catalunya:

- [Entitats jurídiques del Registre d’entitats de Catalunya](https://analisi.transparenciacatalunya.cat/Legislaci-just-cia/Entitats-jur-diques-del-Registre-d-entitats-de-Cat/y6fz-g3ff/about_data)
- [Òrgans de govern de les entitats jurídiques del Registre d’entitats de Catalunya](https://analisi.transparenciacatalunya.cat/es/Legislaci-just-cia/-rgans-de-govern-de-les-entitats-jur-diques-del-Re/i46c-ebpp/about_data)

Per més detalls, consulteu:

- Sobre com s'han preparat les dades per a l'anàlisis: [Estadistiques-AFAs](https://github.com/csala/Estadistiques-AFAs)
- Sobre l'autor de l'anàlisis: [Carles Sala](https://www.linkedin.com/in/carlessalacladellas/) / [GitHub](https://github.com/csala/)

In [1]:
## Preparació de dades

import pandas as pd

data = pd.read_csv('carrecs_afas_amb_genere.csv')

In [2]:
# Funcions d'ajuda

def group_by(data, column, function):
    return data.groupby(column).apply(function).sort_values("dona", ascending=False)

def reindex(series):
    return series.reindex(["dona", "home", "indeterminat"]).fillna(0.0)

# Percentatges Totals

## Global

In [3]:
def percentatge(data):
    return reindex(data.groupby("GENERE").size() / len(data)).round(3)

percentatge(data)

GENERE
dona            0.786
home            0.206
indeterminat    0.008
dtype: float64

## Per Provincia

In [4]:
group_by(data, "NOM_PROVINCIA", percentatge)

GENERE,dona,home,indeterminat
NOM_PROVINCIA,,,
Tarragona,0.806,0.186,0.008
Barcelona,0.784,0.208,0.008
Girona,0.780,0.208,0.012
Lleida,0.773,0.218,0.009


## Per Comarca

In [5]:
group_by(data, "NOM_COMARCA", percentatge)

GENERE,dona,home,indeterminat
NOM_COMARCA,,,
Lluçanès,0.940,0.060,0.000
Montsià,0.890,0.099,0.010
Pallars Jussà,0.883,0.106,0.011
Baix Penedès,0.871,0.118,0.011
Ripollès,0.861,0.127,0.013
Baix Ebre,0.835,0.160,0.005
Pallars Sobirà,0.833,0.167,0.000
Moianès,0.829,0.162,0.010
Baix Empordà,0.825,0.175,0.000


# Estadístiques per AFA

- Percentatge de dones
- 100% femenines
- 100% masculines
- equitatives (40-60%)
- Més dones que homes (>50%)
- Percentatge de dones presidentes

In [6]:
# Agrupat per AFA
per_afa = group_by(data, ["ID_ENTITAT", "NOM_COMARCA", "NOM_PROVINCIA"], percentatge)
per_afa = per_afa.reset_index(level=["NOM_COMARCA", "NOM_PROVINCIA"])

# Genère del càrrec de president/a
presidencia = data[data.CARREC.str.contains("President")].set_index("ID_ENTITAT")["GENERE"]
presidencia = presidencia.reset_index().drop_duplicates()
presidencia = presidencia.set_index("ID_ENTITAT")["GENERE"]
per_afa["presidencia"] = presidencia

def composicions(data):
    values = pd.Series({
        "% de dones": data.dona.mean(),
        "100% dones": (data.dona == 1.0).mean(),
        "100% homes": (data.home == 1.0).mean(),
        "equitatives": data.dona.between(0.4, 0.6).mean(),
        "més dones que homes": (data.dona > 0.5).mean(),
        "% dones presidentes": (data.presidencia == "dona").mean(),
    }).round(3)
    values["Nombre d'AFAs"] = str(len(data))
    return values

## Global

In [7]:
composicions(per_afa) #.round(2).astype(str).str[0:4]

% de dones             0.788
100% dones             0.379
100% homes             0.011
equitatives            0.109
més dones que homes    0.859
% dones presidentes    0.747
Nombre d'AFAs           3788
dtype: object

## Per Provincia

In [8]:
per_afa.reset_index().groupby("NOM_PROVINCIA").apply(composicions).sort_values("100% dones", ascending=False)

,% de dones,100% dones,100% homes,equitatives,més dones que homes,% dones presidentes,Nombre d'AFAs
NOM_PROVINCIA,,,,,,,
Girona,0.789,0.474,0.009,0.066,0.853,0.733,468
Barcelona,0.784,0.380,0.012,0.110,0.848,0.747,2475
Tarragona,0.811,0.351,0.004,0.128,0.892,0.779,453
Lleida,0.782,0.293,0.010,0.130,0.898,0.722,392


In [9]:
per_afa.reset_index().groupby("NOM_COMARCA").apply(composicions).sort_values("100% dones", ascending=False)

,% de dones,100% dones,100% homes,equitatives,més dones que homes,% dones presidentes,Nombre d'AFAs
NOM_COMARCA,,,,,,,
Lluçanès,0.957,0.818,0.000,0.000,1.000,1.000,11
Moianès,0.863,0.619,0.000,0.048,0.857,0.857,21
Alta Ribagorça,0.700,0.600,0.200,0.200,0.600,0.800,5
Ripollès,0.866,0.545,0.000,0.000,1.000,0.818,22
Baix Empordà,0.819,0.526,0.013,0.000,0.897,0.795,78
Selva,0.816,0.524,0.024,0.073,0.878,0.768,82
Alt Empordà,0.812,0.515,0.010,0.087,0.883,0.777,103
Baix Penedès,0.862,0.475,0.000,0.050,0.875,0.825,40
Pallars Jussà,0.889,0.467,0.000,0.000,1.000,0.733,15
